### Tải dữ liệu từ wikipedia về

In [ ]:
!wget https://dumps.wikimedia.org/viwiki/latest/viwiki-latest-pages-articles.xml.bz2

### Giải nén file

In [ ]:
!wikiextractor --json --output output_viwiki --bytes 100M --no-templates viwiki-latest-pages-articles.xml.bz2

### Làm sạch và tiền xử lý dữ liệu

In [20]:
import os
import json
import re
from tqdm import tqdm
import glob2

# Danh sách ký tự hợp lệ trong tiếng Việt
intab_l = "ạảãàáâậầấẩẫăắằặẳẵóòọõỏôộổỗồốơờớợởỡéèẻẹẽêếềệểễúùụủũưựữửừứíìịỉĩýỳỷỵỹđ"
ascii_lowercase = 'abcdefghijklmnopqrstuvwxyz'
digits = '0123456789'
punctuation = r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""
whitespace = ' '
accept_strings = intab_l + ascii_lowercase + digits + punctuation + whitespace
char_pattern = re.compile('^[' + re.escape(accept_strings) + ']+$')

def _check_tieng_viet(seq: str) -> bool:
    return bool(re.match(char_pattern, seq.lower()))

In [21]:
def process_wiki_folder(input_folder: str, output_file: str):
    idx = 0
    all_files = glob2.glob(os.path.join(input_folder, '**'), recursive=True)

    with open(output_file, 'w', encoding='utf8') as out_f:
        for path in tqdm(all_files, desc="Đang xử lý file Wikipedia"):
            if os.path.isfile(path):
                try:
                    with open(path, 'r', encoding='utf8') as f:
                        for line in f:
                            try:
                                art_json = json.loads(line)
                                art_cont = art_json.get('text', '')
                                art_cont = re.sub(r"\s+", " ", art_cont)  # Gom khoảng trắng dư
                                art_seqs = art_cont.split("\n")  # Tách thành từng câu
                                for seq in art_seqs:
                                    if _check_tieng_viet(seq) and len(seq.strip()) > 20:
                                        idx_str = str(idx).zfill(10)
                                        out_f.write(f"{idx_str}\t{seq.strip()}\n")
                                        idx += 1
                            except json.JSONDecodeError:
                                continue
                except Exception as e:
                    print(f"Lỗi đọc {path}: {e}")

    print(f"\nTổng số dòng hợp lệ: {idx} dòng. Ghi vào '{output_file}'")

# 📂 Ví dụ sử dụng
if __name__ == "__main__":
    input_folder = r'C:\Users\tontide1\my-data\Vietnamese-Accent-Model\output_viwiki\AA'
    output_file = 'train_tieng_viet.txt'
    process_wiki_folder(input_folder, output_file)

Đang xử lý file Wikipedia: 100%|██████████| 20/20 [01:15<00:00,  3.75s/it]


Tổng số dòng hợp lệ: 1036774 dòng. Ghi vào 'train_tieng_viet.txt'
